In [1]:
#import libraries
import pickle
from collections import defaultdict
from urllib.request import urlopen
import numpy as np
import pandas as pd
import os
import h5py
import sys
import requests
import zipfile
import inspect

In [2]:
#download data

#file located here: https://drive.google.com/file/d/1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P/view

#make data directory if doesn't exist in path folder
os.makedirs('data', exist_ok=True)

#download meld and mosei zip files for data (mosei is from CMU, meld is friends episodes)
if('mosei' in  os.listdir('data')):
    pass
else:
    !file=1tcVYIMcZdlDzGuJvnMtbMchKIK9ulW1P && wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='${file} -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="${file} -O data/data.zip && rm -rf /tmp/cookies.txt

    #unzip data
    !unzip data/data.zip -d data/mosei

In [3]:
#preprocess data
from mtl.preprocess import clean_text, create_embedding_matrix, data_processing

X_train, X_test, X_train_pad, X_test_pad, y_train, y_test, y_train1, y_test1, y_train2, y_test2, y_train3, y_test3, y_train4, y_test4, y_train5, y_test5, y_train6, y_test6, embedd_matrix, vocab_size, X_val, X_val_pad, y_val, y_val1, y_val2, y_val3, y_val4, y_val5, y_val6, data_test = data_processing()

y_mtl_train = (y_train, y_train1, y_train2, y_train3, y_train4, y_train5, y_train6 )
y_mtl_val = (y_val, y_val1, y_val2, y_val3, y_val4, y_val5, y_val6)

y_emo_train = (y_train1, y_train2, y_train3, y_train4, y_train5, y_train6 )
y_emo_val = (y_val1, y_val2, y_val3, y_val4, y_val5, y_val6)


[nltk_data] Downloading package punkt to /home/eileen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Training data sentiment statistics
4.0    4120
3.0    3000
2.0    2040
5.0    1549
1.0    1305
Name: sentiment, dtype: int64
Test data sentiment statistics
3.0    1927
4.0    1230
2.0     554
5.0     441
1.0     399
Name: sentiment, dtype: int64
Validation data sentiment statistics
3.0    835
4.0    470
2.0    244
5.0    169
1.0    110
Name: sentiment, dtype: int64
[ 'hi ' 'umm ' 'today ' 'the ' 'product ' 'that ' `` i 'm '' 'going ' 'to ' 'review' 'is ' ' a ' 'sp ' 'this ' 'is ' ]
Number of unique words: 18313
{'1.0': 0, '2.0': 1, '3.0': 2, '4.0': 3, '5.0': 4}
Words found in wiki vocab: 651
New words found: 17662


In [4]:
#train

from mtl.network import net
from mtl.train import train_model
from tensorboard.plugins.hparams import api as hp
import datetime

model_name = 'mtl_five_sentiments'
saved_model_name = 'models/'+ model_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5'
print('models/'+ model_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5')
batch_size=32
epochs=2
gru_output_size=128
dropout=0.2
recurrent_dropout=0.2
tensorboard = False
loss_weights = [0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
print(vocab_size)


train_model(vocab_size = vocab_size, embedd_matrix = embedd_matrix, data_x = X_train_pad, data_y = y_mtl_train, val_x = X_val_pad, val_y = y_mtl_val, model_name=model_name, saved_model_name=saved_model_name, batch_size=32, epochs=2, tensorboard=tensorboard, loss_weights=loss_weights)

models/mtl_five_sentiments20220404-124243.h5
18314
Training a mtl_five_sentiments model!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 300)     5494200     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 256)          330240      ['embedding[0][0]']              
                                                                                                  
 s (Dense)                      (None, 5)            1285        ['bidirectional[0][0]']          
     

In [ ]:
#predict